# Notebook setup

In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import numpy as np
from scipy import sparse

In [ ]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

# Contribution analysis

Contribution analysis means different things to different people - here we look at the processes whose direct emissions were causing the damage that we assessed, regardless of where they occur in the supply chain.

In [ ]:
functional_unit = bd.Database("ecoinvent-3.12-cutoff").random()
impact_category = bd.methods.random()

functional_unit, impact_category

In [ ]:
lca = bc.LCA({functional_unit: 1}, method=impact_category)
lca.lci()
lca.lcia()
lca.score

The easiest way to do such contribution analysis is with the `to_dataframe()` method:

In [ ]:
lca.to_dataframe()

This has a lot of input parameters:

In [ ]:
lca.to_dataframe?

Here are the matrices in conventional Brightway usage:

In [ ]:
(
    lca.technosphere_matrix.shape,
    lca.biosphere_matrix.shape,
    lca.characterization_matrix.shape,
    lca.inventory.shape,
    lca.characterized_inventory.shape,
)

# Miscellaneous functions

## Graphing the technosphere matrix

In [ ]:
from bw2analyzer.matrix_grapher import SparseMatrixGrapher 

In [ ]:
smg = SparseMatrixGrapher(lca.technosphere_matrix).ordered_graph(width=8, height=8)

In [ ]:
smg = SparseMatrixGrapher(lca.biosphere_matrix).graph(width=8, height=8)

## Economic analysis

Do the rich activities have all the impact in this world?

In [ ]:
from bw2analyzer.econ import gini_coefficient, herfindahl_index, concentration_ratio, theil_index

In [ ]:
income = lca.characterized_inventory.data

In [ ]:
gini_coefficient(income), herfindahl_index(income), concentration_ratio(income), theil_index(income)

# Traversing the supply chain

There is an industrial-strength class in `bw2calc` called `GraphTraversal`, but for quick analysis we can use the following:

In [ ]:
ba.print_recursive_supply_chain(functional_unit, max_level=2)

In the above function, you can play with the following default arguments:

* amount=1,
* max_level=2,
* cutoff=0,
* file_obj=None,
* tab_character="  ",
* level=0,

Similarly, we can traverse the *characterized* inventory.

In [ ]:
ba.print_recursive_calculation(
    functional_unit, 
    impact_category
)

Same thing here...